In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import networkx as nx
import os

In [2]:
mat_data= scipy.io.loadmat('dataMSc.mat')
data= mat_data['data']

In [3]:
data.shape

(1, 98)

In [4]:
data[0,11][0,1].shape #64 channels and 3 contrasts

(64, 64, 3)

In [5]:
data[0,11][0,1][:,:,2] # 12th subject, 2nd run, HbT

array([[ 0.        ,  0.11291833,  0.55476964, ...,         nan,
        -0.11134297,         nan],
       [ 0.11291833,  0.        ,  0.19523854, ...,         nan,
        -0.11484919,         nan],
       [ 0.55476964,  0.19523854,  0.        , ...,         nan,
        -0.21097138,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [-0.11134297, -0.11484919, -0.21097138, ...,         nan,
         0.        ,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]])

## Fisher's r-to-z tranformation for converting to normal distribution:

In [6]:
def fisher_r_to_z(r):
    return np.arctanh(r)

In [7]:
num_subjects = data.shape[1]
num_contrasts = 3  # HbO, HbR, HbT
num_channels = 64

In [8]:
average_correlation_matrices = np.zeros((num_subjects, num_channels, num_channels, num_contrasts))

In [9]:
for i in range(num_subjects): # range will return a list from 0 to 97, hence i-1 will be adjusted to i
    subject_data = data[0, i]
    num_runs = len(subject_data[0])
    
    if num_runs == 0:
        continue  # No data for this subject, 'continue' is used to start the next iteration

    for contrast in range(num_contrasts):
        z_sum = np.zeros((num_channels, num_channels))
        count = np.zeros((num_channels, num_channels))
        
        for run in range(num_runs):
            run_data = subject_data[0, run]
            
            if run_data.size == 0:
                continue  # Empty run data
            
            correlation_matrix = run_data[:, :, contrast]
            valid = ~np.isnan(correlation_matrix) # a mask to identify indices with not (~) NaN values
            
            z_values = np.zeros_like(correlation_matrix)
            z_values[valid] = fisher_r_to_z(correlation_matrix[valid])
            
            z_sum[valid] += z_values[valid]
            count[valid] += 1
        
        # Avoid division by zero
        valid_counts = count > 0
        z_avg = np.zeros_like(z_sum)
        z_avg[valid_counts] = z_sum[valid_counts] / count[valid_counts]
        
        average_correlation_matrices[i, :, :, contrast] = z_avg

In [10]:
average_correlation_matrices.shape

(98, 64, 64, 3)

# Calculating topological measures and contructing brain graphs

In [12]:
correlation_threshold = 0.3
topological_measures_list = []

for i in range(11, num_subjects):
    # Use the HbT contrast for simplicity
    correlation_matrix = average_correlation_matrices[i, :, :, 2]
    
    # Create a graph from the correlation matrix
    G = nx.Graph()
    
    for j in range(num_channels):
        for k in range(j+1, num_channels):
            if correlation_matrix[j, k] > correlation_threshold:
                G.add_edge(j, k, weight=correlation_matrix[j, k])
    
    # Compute topological measures
    degree = dict(G.degree())
    clustering_coefficient = nx.clustering(G, weight='weight')
    betweenness_centrality = nx.betweenness_centrality(G, weight='weight')
    
    # Check if the graph is not empty before calculating efficiency
    if len(G) > 0:
        global_efficiency = nx.global_efficiency(G)
        local_efficiency = nx.local_efficiency(G)
    else:
        global_efficiency = np.nan
        local_efficiency = np.nan
    
    # Compute the strength of each node
    strength = {node: sum(data['weight'] for _, data in G[node].items()) for node in G.nodes}
    
    # Calculate the degree density for each node
    N = len(G.nodes)
    degree_density = {node: degree[node] / (N - 1) for node in G.nodes}

    # Store the measures in a dictionary
    measures = {
        'subject': i + 1,
        'global_efficiency': global_efficiency,
        'local_efficiency': local_efficiency,
        'degree': degree,
        'degree_density': degree_density,
        'clustering_coefficient': clustering_coefficient,
        'betweenness_centrality': betweenness_centrality,
        'strength': strength
    }
    topological_measures_list.append(measures)
    
    # Plot the graph
    plt.figure(figsize=(8, 8))
    pos = nx.spring_layout(G)  # Use spring layout for better visualization
    nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', edge_color='gray')
    plt.title(f'Subject {i+1}')
    
    # Save the graph to a file
    graph_path = os.path.join('/Users/paruljain/Desktop/MS Data Science/Dissertation/Brain networks/', f'subject_{i+1}_graph.png')
    plt.savefig(graph_path)
    plt.close()

In [13]:
anatomical_pos= pd.read_csv('Anatomical Positions of the fNIRS Channels.txt')
anatomical_pos

,Channel,Label_name,\tch_coord_(Monte_Carlo)\t\t
0,1,Cuneus_L,\t\t188\t102\t140
1,2,Cuneus_L,\t\t199\t95\t135
2,3,Occipital_Sup_L,\t184\t90\t141
3,4,Cuneus_L,\t\t181\t87\t134
4,5,Frontal_Sup_Medial_L,\t152\t222\t129
...,...,...,...
59,60,Frontal_Inf_Oper_R,\t160\t187\t69
60,61,Rolandic_Oper_R,\t169\t163\t64
61,62,Angular_R,\t\t183\t135\t89
62,63,Temporal_Mid_R,\t176\t137\t83


In [14]:
anatomical_pos.drop(columns='\tch_coord_(Monte_Carlo)\t\t',inplace=True)

In [15]:
anatomical_pos

,Channel,Label_name
0,1,Cuneus_L
1,2,Cuneus_L
2,3,Occipital_Sup_L
3,4,Cuneus_L
4,5,Frontal_Sup_Medial_L
...,...,...
59,60,Frontal_Inf_Oper_R
60,61,Rolandic_Oper_R
61,62,Angular_R
62,63,Temporal_Mid_R


In [16]:
# Create a dictionary from the mapping DataFrame
channel_to_label = dict(zip(anatomical_pos['Channel'] - 1, anatomical_pos['Label_name']))  # Adjust for zero-based index

In [17]:
loc_lobes_df= pd.read_csv('fNIRS channels anatomical labels.csv')
loc_lobes_df

,Channel,Label_name,Lobe,Hemisphere,Cortex_Location
0,1,Cuneus_L,Occipital,L,Occipital Cortex
1,2,Cuneus_L,Occipital,L,Occipital Cortex
2,3,Occipital_Sup_L,Occipital,L,Occipital Cortex
3,4,Cuneus_L,Occipital,L,Occipital Cortex
4,5,Frontal_Sup_Medial_L,Frontal,L,Prefrontal Cortex
...,...,...,...,...,...
59,60,Frontal_Inf_Oper_R,Frontal,R,Prefrontal Cortex
60,61,Rolandic_Oper_R,Parietal,R,Somatosensory Cortex
61,62,Angular_R,Parietal,R,Parietal Cortex
62,63,Temporal_Mid_R,Temporal,R,Temporal Cortex


In [18]:
# Create a dictionary from the mapping DataFrame
channel_to_lobe = dict(zip(loc_lobes_df['Channel'] - 1, loc_lobes_df['Lobe']))  # Adjust for zero-based index

In [19]:
# Create a dictionary from the mapping DataFrame
channel_to_hem = dict(zip(loc_lobes_df['Channel'] - 1, loc_lobes_df['Hemisphere']))  # Adjust for zero-based index

# Properties at Microscopic level

In [20]:
# Create a DataFrame to store the topological measures in a tabular format
rows = []
for measures in topological_measures_list:
    subject = measures['subject']
    global_efficiency = measures['global_efficiency']
    local_efficiency = measures['local_efficiency']
    degree = measures['degree']
    degree_density = measures['degree_density']
    clustering_coefficient = measures['clustering_coefficient']
    betweenness_centrality = measures['betweenness_centrality']
    strength = measures['strength']
    
    for node in range(num_channels):
        row = {
            'subject': subject,
            'node': node,
            'global_efficiency': global_efficiency,
            'local_efficiency': local_efficiency,
            'degree': degree.get(node, np.nan),
            'degree_density': degree_density.get(node, np.nan),
            'clustering_coefficient': clustering_coefficient.get(node, np.nan),
            'betweenness_centrality': betweenness_centrality.get(node, np.nan),
            'strength': strength.get(node, np.nan),
            'anatomical_location': channel_to_label.get(node),
            'lobe': channel_to_lobe.get(node),
            'hemisphere': channel_to_hem.get(node)
        }
        rows.append(row)

df = pd.DataFrame(rows)

In [21]:
df

,subject,node,global_efficiency,local_efficiency,degree,degree_density,clustering_coefficient,betweenness_centrality,strength,anatomical_location,lobe,hemisphere
0,12,0,0.260726,0.552694,4.0,0.125000,0.081554,0.131048,1.401722,Cuneus_L,Occipital,L
1,12,1,0.260726,0.552694,1.0,0.031250,0.000000,0.000000,0.365050,Cuneus_L,Occipital,L
2,12,2,0.260726,0.552694,1.0,0.031250,0.000000,0.000000,0.353318,Occipital_Sup_L,Occipital,L
3,12,3,0.260726,0.552694,4.0,0.125000,0.327706,0.032258,1.779783,Cuneus_L,Occipital,L
4,12,4,0.260726,0.552694,6.0,0.187500,0.228695,0.050403,2.409319,Frontal_Sup_Medial_L,Frontal,L
...,...,...,...,...,...,...,...,...,...,...,...,...
5563,98,59,0.457431,0.504721,4.0,0.121212,0.069127,0.049242,1.790528,Frontal_Inf_Oper_R,Frontal,R
5564,98,60,0.457431,0.504721,3.0,0.090909,0.180129,0.007576,1.399714,Rolandic_Oper_R,Parietal,R
5565,98,61,0.457431,0.504721,6.0,0.181818,0.206329,0.113636,2.217607,Angular_R,Parietal,R
5566,98,62,0.457431,0.504721,4.0,0.121212,0.089372,0.058712,1.822868,Temporal_Mid_R,Temporal,R


In [22]:
df.isna().sum()

subject                      0
node                         0
global_efficiency          448
local_efficiency           448
degree                    2151
degree_density            2151
clustering_coefficient    2151
betweenness_centrality    2151
strength                  2151
anatomical_location          0
lobe                         0
hemisphere                   0
dtype: int64

In [23]:
df.dropna(inplace=True)
df

,subject,node,global_efficiency,local_efficiency,degree,degree_density,clustering_coefficient,betweenness_centrality,strength,anatomical_location,lobe,hemisphere
0,12,0,0.260726,0.552694,4.0,0.125000,0.081554,0.131048,1.401722,Cuneus_L,Occipital,L
1,12,1,0.260726,0.552694,1.0,0.031250,0.000000,0.000000,0.365050,Cuneus_L,Occipital,L
2,12,2,0.260726,0.552694,1.0,0.031250,0.000000,0.000000,0.353318,Occipital_Sup_L,Occipital,L
3,12,3,0.260726,0.552694,4.0,0.125000,0.327706,0.032258,1.779783,Cuneus_L,Occipital,L
4,12,4,0.260726,0.552694,6.0,0.187500,0.228695,0.050403,2.409319,Frontal_Sup_Medial_L,Frontal,L
...,...,...,...,...,...,...,...,...,...,...,...,...
5562,98,58,0.457431,0.504721,3.0,0.090909,0.358873,0.007576,1.450415,Precentral_R,Frontal,R
5563,98,59,0.457431,0.504721,4.0,0.121212,0.069127,0.049242,1.790528,Frontal_Inf_Oper_R,Frontal,R
5564,98,60,0.457431,0.504721,3.0,0.090909,0.180129,0.007576,1.399714,Rolandic_Oper_R,Parietal,R
5565,98,61,0.457431,0.504721,6.0,0.181818,0.206329,0.113636,2.217607,Angular_R,Parietal,R


In [24]:
df.to_csv('microscopic_topological measures.csv',index=False)

# Mesoscopic Properties

In [29]:
df

,subject,node,global_efficiency,local_efficiency,degree,clustering_coefficient,betweenness_centrality,strength,anatomical_location,lobe,hemisphere
0,12,0,0.260726,0.552694,4.0,0.081554,0.131048,1.401722,Cuneus_L,Occipital,L
1,12,1,0.260726,0.552694,1.0,0.000000,0.000000,0.365050,Cuneus_L,Occipital,L
2,12,2,0.260726,0.552694,1.0,0.000000,0.000000,0.353318,Occipital_Sup_L,Occipital,L
3,12,3,0.260726,0.552694,4.0,0.327706,0.032258,1.779783,Cuneus_L,Occipital,L
4,12,4,0.260726,0.552694,6.0,0.228695,0.050403,2.409319,Frontal_Sup_Medial_L,Frontal,L
...,...,...,...,...,...,...,...,...,...,...,...
5562,98,58,0.457431,0.504721,3.0,0.358873,0.007576,1.450415,Precentral_R,Frontal,R
5563,98,59,0.457431,0.504721,4.0,0.069127,0.049242,1.790528,Frontal_Inf_Oper_R,Frontal,R
5564,98,60,0.457431,0.504721,3.0,0.180129,0.007576,1.399714,Rolandic_Oper_R,Parietal,R
5565,98,61,0.457431,0.504721,6.0,0.206329,0.113636,2.217607,Angular_R,Parietal,R


In [30]:
# Mesoscopic (Anatomical Location Level) Average Metrics
mesoscopic_avg = df.groupby(['lobe','hemisphere']).mean().reset_index()
mesoscopic_avg

,lobe,hemisphere,subject,node,global_efficiency,local_efficiency,degree,clustering_coefficient,betweenness_centrality,strength
0,Frontal,L,56.149688,13.814969,0.454476,0.686769,9.591476,0.241022,0.041378,4.733044
1,Frontal,R,55.714773,45.723864,0.457167,0.690730,9.211364,0.243982,0.039501,4.567636
2,Occipital,L,54.320911,9.884058,0.450565,0.691312,8.293996,0.267092,0.036632,4.496361
3,Occipital,R,53.723164,39.242938,0.454570,0.689259,8.559322,0.256488,0.037720,4.543073
4,Parietal,L,53.768939,24.897727,0.445539,0.670366,7.348485,0.194935,0.040173,3.349679
5,Parietal,R,54.661654,57.706767,0.457096,0.683102,8.150376,0.172222,0.039351,3.772155
6,Temporal,L,53.560748,30.457944,0.461339,0.684605,7.121495,0.182530,0.039134,3.265652
7,Temporal,R,52.554455,62.435644,0.461251,0.680941,6.821782,0.160695,0.031708,3.174022


In [31]:
mesoscopic_avg.drop(columns='subject',inplace=True)

In [32]:
mesoscopic_avg

,lobe,hemisphere,node,global_efficiency,local_efficiency,degree,clustering_coefficient,betweenness_centrality,strength
0,Frontal,L,13.814969,0.454476,0.686769,9.591476,0.241022,0.041378,4.733044
1,Frontal,R,45.723864,0.457167,0.690730,9.211364,0.243982,0.039501,4.567636
2,Occipital,L,9.884058,0.450565,0.691312,8.293996,0.267092,0.036632,4.496361
3,Occipital,R,39.242938,0.454570,0.689259,8.559322,0.256488,0.037720,4.543073
4,Parietal,L,24.897727,0.445539,0.670366,7.348485,0.194935,0.040173,3.349679
5,Parietal,R,57.706767,0.457096,0.683102,8.150376,0.172222,0.039351,3.772155
6,Temporal,L,30.457944,0.461339,0.684605,7.121495,0.182530,0.039134,3.265652
7,Temporal,R,62.435644,0.461251,0.680941,6.821782,0.160695,0.031708,3.174022


In [40]:
mesoscopic_avg.to_csv('mesoscopic_properties.csv',index=False)

# Macroscopic Properties

In [25]:
# Macroscopic (Subject Level) Average Metrics
macroscopic_avg = df.groupby('subject').mean().reset_index()

In [26]:
#macroscopic_avg.drop(columns='node',inplace=True)
macroscopic_avg

,subject,node,global_efficiency,local_efficiency,degree,degree_density,clustering_coefficient,betweenness_centrality,strength
0,12,25.757576,0.260726,0.552694,3.575758,0.111742,0.228683,0.032991,1.668256
1,13,30.468085,0.320154,0.658336,4.297872,0.093432,0.252247,0.078199,2.134799
2,14,33.740741,0.481761,0.716887,8.296296,0.156534,0.159564,0.030277,4.070918
3,15,31.744186,0.483861,0.582839,8.279070,0.197121,0.141935,0.030116,3.720559
4,16,30.762712,0.454432,0.688972,8.644068,0.149036,0.245438,0.031848,3.993715
...,...,...,...,...,...,...,...,...,...
75,94,29.833333,0.480026,0.713423,7.888889,0.225397,0.234156,0.037862,4.269881
76,95,30.946429,0.615184,0.775684,16.571429,0.301299,0.248870,0.018086,7.832750
77,96,28.957447,0.433210,0.560206,6.000000,0.130435,0.183461,0.043643,2.981635
78,97,31.157895,0.456700,0.647276,8.350877,0.149123,0.182259,0.034165,4.169841


In [27]:
macroscopic_avg

,subject,node,global_efficiency,local_efficiency,degree,degree_density,clustering_coefficient,betweenness_centrality,strength
0,12,25.757576,0.260726,0.552694,3.575758,0.111742,0.228683,0.032991,1.668256
1,13,30.468085,0.320154,0.658336,4.297872,0.093432,0.252247,0.078199,2.134799
2,14,33.740741,0.481761,0.716887,8.296296,0.156534,0.159564,0.030277,4.070918
3,15,31.744186,0.483861,0.582839,8.279070,0.197121,0.141935,0.030116,3.720559
4,16,30.762712,0.454432,0.688972,8.644068,0.149036,0.245438,0.031848,3.993715
...,...,...,...,...,...,...,...,...,...
75,94,29.833333,0.480026,0.713423,7.888889,0.225397,0.234156,0.037862,4.269881
76,95,30.946429,0.615184,0.775684,16.571429,0.301299,0.248870,0.018086,7.832750
77,96,28.957447,0.433210,0.560206,6.000000,0.130435,0.183461,0.043643,2.981635
78,97,31.157895,0.456700,0.647276,8.350877,0.149123,0.182259,0.034165,4.169841


In [28]:
macroscopic_avg.to_csv('macroscopic_properties.csv',index=False)